In [1]:
import os

In [2]:
%pwd

'e:\\Wine_Data_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Wine_Data_Project'

# Create the entity

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Create the configuration

In [13]:
# It will return all the configuration like root dir, URL, ZIP file etc

from ML_Projects.constants import *
from ML_Projects.utils.common import read_yaml, create_directories

# Create the configuartion manager 

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# Create the data ingestion component

In [15]:
import os
import urllib.request as request
import zipfile
import csv
from ML_Projects import logger
from ML_Projects.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Only downloading CSV, no need for extraction
    data_ingestion.download_file()
    
except Exception as e:
    logger.error(f"❌ An error occurred: {e}")
    raise e


[2025-03-06 16:20:28,966: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-06 16:20:28,971: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-06 16:20:28,973: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-06 16:20:28,974: INFO: common: created directory at: artifacts]
[2025-03-06 16:20:28,975: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-06 16:20:29,815: INFO: 2366278714: artifacts/data_ingestion/data.csv download! with following info: 
Connection: close
Content-Length: 87343
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "fdd61bf274bd0a46e8bb4eec3d30d75f86f65715d5e6e0e30b8f2a287bd1833b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4974:3A3BB8:2A47A3:6A3CEA:67C97DB6
Accept-Ranges: bytes
Dat

In [28]:
%pwd

'e:\\Wine_Data_Project\\research'

In [29]:
os.chdir("../")